# 필요 라이브러리 import

In [7]:
import pandas as pd                                          #데이터 분석 라이브러리
from sklearn.decomposition import PCA                        #주성분 분석 라이브러리
import numpy as np                                           # 계산 라이브러리
from sklearn.preprocessing import MinMaxScaler               # 최소 최대 0~1 범위로 변환을 위한 라이브러리

from sklearn.model_selection import train_test_split         # 데이터 분할 라이브러리
from sklearn.model_selection import cross_validate           # Kfold 라이브러리
from sklearn.model_selection import cross_val_score          # Kfold 라이브러리
from sklearn.model_selection import StratifiedKFold          # Kfold 라이브러리


from sklearn.neighbors import KNeighborsClassifier           # KNN 라이브러리
from sklearn.svm import SVC                                  # SVM Classifier 라이브러리
from sklearn.tree import DecisionTreeClassifier              # 결정나무분류 라이브러리
from sklearn.linear_model import LogisticRegression          # 로지스틱 회귀 라이브러리
from bayes_opt import BayesianOptimization                   # 베이시안 라이브러리   


from sklearn.metrics import make_scorer                      # 평가용 라이브러리
from sklearn.metrics import f1_score                         # f1  라이브러리
from sklearn.metrics import accuracy_score                   # accuracy 라이브러리 



import matplotlib.pyplot as plt                              # 그래프 라이브러리
from sklearn.ensemble import VotingClassifier                # 보팅 앙상블 라이브러리
import seaborn as sns                                        # boxplot 을 위한 라이브러리

# data 불러오기

In [8]:
File = pd.read_csv("./Weighted_data_nonscale.csv")

In [9]:
Floodfile = File.drop(['Unnamed: 0','FLOOD'], axis = 1)
Floodclass = File['FLOOD']

In [10]:
Floodfile

,PUMP_RATIO,Hourly_Rainfall_Weight,Impervious_Surface_Weight,SLOPE_AVG_Weight,HIGH_Weight,F_WEIGHT
0,4.115226e-07,0.104348,0.467342,0.599952,0.045968,0.075019
1,4.115226e-07,0.208696,0.467342,0.599952,0.045968,0.075019
2,4.115226e-07,0.259722,0.467342,0.599952,0.045968,0.075019
3,4.115226e-07,0.010870,0.467342,2.699784,0.072218,0.075019
4,4.115226e-07,0.108696,0.467342,2.699784,0.072218,0.075019
...,...,...,...,...,...,...
791,1.364381e-08,0.573248,1.156156,7.040000,115.950324,0.016667
792,1.364381e-08,0.573248,1.156156,21.608643,13.159136,0.016667
793,1.364381e-08,0.573248,1.156156,12.147570,0.066817,0.016667
794,1.364381e-08,0.573248,1.156156,21.608643,0.061794,0.016667


# minmaxscaler
## data scale 과정

In [11]:
scaler = MinMaxScaler()
scaler.fit(Floodfile)
Floodfile = scaler.transform(Floodfile)

# PCA

In [12]:
pca = PCA(n_components=4) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(Floodfile)
Floodfile = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2','principal component3', 'principal component4'])

In [13]:
Floodfile

,principal component1,principal component2,principal component3,principal component4
0,-0.011940,0.141868,-0.346867,0.120298
1,-0.034011,0.137298,-0.260972,-0.035119
2,-0.044803,0.135063,-0.218968,-0.111119
3,-0.003553,0.115147,-0.344956,0.306535
4,-0.024245,0.110863,-0.264429,0.160832
...,...,...,...,...
791,-0.459173,0.036658,0.376578,-0.516097
792,-0.532985,-0.169149,0.914837,-0.197536
793,-0.481033,-0.029302,0.558410,-0.410303
794,-0.532326,-0.168134,0.913709,-0.198502


# data split
## 8 : 2 으로 training 과 test 구분

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Floodfile, Floodclass , test_size=0.2, random_state=400)

In [ ]:
skfolds=StratifiedKFold(n_splits=3, random_state=400 ,shuffle = True)

# BayesianOptimization

## KNN

In [ ]:
KNN_search_point = {'k': (3,50), 'leaf': (1,50)}

gp_params = {"alpha":1e-10}
scores_all = []
k_all = []
leaf_all=[]

def KNN_opt(k,leaf):
    cvscores = []
    
    for train_index, val_index in skfolds.split(x_train, y_train):
#         print(train_index)
#         print(val_index)
        x_skftrain, x_val = x_train.values[train_index], x_train.values[val_index]
        Y_skftrain, Y_val = np.array(y_train)[train_index],np.array(y_train)[val_index]
        
        KNNmodel = KNeighborsClassifier(n_neighbors = int(k),weights = 'distance', leaf_size= leaf ) # KNN  
        
        KNNmodel2 = KNNmodel.fit(x_skftrain,Y_skftrain)      # 모델학습
        y_pred = KNNmodel2.predict(x_val)                    # 예측

        report = f1_score(Y_val, y_pred, average='weighted') # F1
#         report = accuracy_score(Y_val, y_pred)             # accuracy
        cvscores.append(report)
    
    
    scores_all.append(np.mean(cvscores))
    k_all.append(int(k))
#     leaf_all.append(int(leaf))
    
    return np.mean(cvscores)

result = BayesianOptimization(KNN_opt, KNN_search_point,    random_state=10,    verbose=2) 
result.maximize(acq='ucb',n_iter=15, kappa=2.576, init_points=1 , **gp_params)

In [ ]:
plt.scatter(k_all, scores_all)

## SVM

In [ ]:
SVM_search_point = {'k': (1,50), 'gamma': (0.001,20)}

gp_params = {"alpha":1e-10}
scores_all = []
k_all = []
gamma_all = []
def SVM_opt(k,gamma):
    cvscores = []
    
    for train_index, val_index in skfolds.split(x_train, y_train):
        x_skftrain, x_val = x_train.values[train_index],x_train.values[val_index]
        Y_skftrain, Y_val = np.array(y_train)[train_index],np.array(y_train)[val_index]

        SVMmodel = SVC( C= int(k) , kernel='rbf' , probability = True, gamma = gamma ) # SVM  
        
        SVMmodel2 = SVMmodel.fit(x_skftrain,Y_skftrain)      # 모델학습
        y_pred = SVMmodel2.predict(x_val)                    # 예측

        report = f1_score(Y_val, y_pred, average='weighted') # F1
#         report = accuracy_score(Y_val, y_pred)             # accuracy
        cvscores.append(report)
    
    scores_all.append(np.mean(cvscores))
    k_all.append(int(k))
    return np.mean(cvscores)

result = BayesianOptimization(SVM_opt, SVM_search_point, random_state=10, verbose=2) 
result.maximize(acq='ucb',n_iter=15, kappa=2.576, init_points=1 , **gp_params)

In [ ]:
SVM_search_point = {'k': (1,50)}

gp_params = {"alpha":1e-10}
scores_all = []
k_all = []
gamma_all = []
def SVM_opt(k):
    cvscores = []
    
    for train_index, val_index in skfolds.split(x_train, y_train):
        x_skftrain, x_val = x_train.values[train_index],x_train.values[val_index]
        Y_skftrain, Y_val = np.array(y_train)[train_index],np.array(y_train)[val_index]

        SVMmodel = SVC( C= int(k) , kernel='poly' , probability = True ) # SVM  
        
        SVMmodel2 = SVMmodel.fit(x_skftrain,Y_skftrain)      # 모델학습
        y_pred = SVMmodel2.predict(x_val)                    # 예측

        report = f1_score(Y_val, y_pred, average='weighted') # F1
#         report = accuracy_score(Y_val, y_pred)             # accuracy
        cvscores.append(report)
    
    scores_all.append(np.mean(cvscores))
    k_all.append(int(k))
    return np.mean(cvscores)

result = BayesianOptimization(SVM_opt, SVM_search_point, random_state=10, verbose=2) 
result.maximize(acq='ucb',n_iter=15, kappa=2.576, init_points=1 , **gp_params)

In [ ]:
plt.scatter(k_all, scores_all)

## Decision Tree

In [ ]:
DT_search_point = { 'k': (1,30), 'leaf' : ( 2, 30), 'split': (2,30) }

gp_params = {"alpha":1e-10}
scores_all = []
k_all = []
leaf_all = []
split_all = []

def DT_opt(k,leaf,split):
    cvscores = []
    
    for train_index, val_index in skfolds.split(x_train, y_train):
        x_skftrain, x_val = x_train.values[train_index],x_train.values[val_index]
        Y_skftrain, Y_val = np.array(y_train)[train_index],np.array(y_train)[val_index]

        DTmodel = DecisionTreeClassifier(criterion='gini',
                                         max_depth=int(k),              # 트리의 최대 깊이
                                         min_samples_leaf= int(leaf),
                                         min_samples_split= int(split)) #DT  
        
        
        DTmodel2 = DTmodel.fit(x_skftrain,Y_skftrain)      # 모델학습
        y_pred = DTmodel2.predict(x_val)                    # 예측

        report = f1_score(Y_val, y_pred, average='weighted') # F1
#         report = accuracy_score(Y_val, y_pred)             # accuracy
        cvscores.append(report)
    
    scores_all.append(np.mean(cvscores))
    k_all.append(int(k))
    leaf_all.append(int(leaf))
    split_all.append(split)

    
    return np.mean(cvscores)

result = BayesianOptimization(DT_opt, DT_search_point, random_state=10, verbose=2) 
result.maximize(acq='ucb',n_iter=15, kappa=2.576, init_points=1 , **gp_params)

In [ ]:
plt.scatter(k_all, scores_all)

## Logistic Regression

In [ ]:
LR_search_point = { 'k': (1,50)}

gp_params = {"alpha":1e-10}
scores_all = []
k_all = []

def LR_opt(k):
    cvscores = []
    
    for train_index, val_index in skfolds.split(x_train, y_train):
        x_skftrain, x_val = x_train.values[train_index],x_train.values[val_index]
        Y_skftrain, Y_val = np.array(y_train)[train_index],np.array(y_train)[val_index]

        LRmodel = LogisticRegression(C= int(k) , multi_class= 'auto') #LR  
        
        
        LRmodel2 = LRmodel.fit(x_skftrain,Y_skftrain)      # 모델학습
        y_pred = LRmodel2.predict(x_val)                    # 예측

        report = f1_score(Y_val, y_pred, average='weighted') # F1
#         report = accuracy_score(Y_val, y_pred)             # accuracy
        cvscores.append(report)
    
    scores_all.append(np.mean(cvscores))
    k_all.append(int(k))
    
    return np.mean(cvscores)

result = BayesianOptimization(LR_opt, LR_search_point, random_state=10, verbose=2) 
result.maximize(acq='ucb',n_iter=15, kappa=2.576, init_points=1 , **gp_params)

In [ ]:
plt.scatter(k_all, scores_all)

# Voting ensemble
## 사용 된 모델 : KNN, SVM, Decision Tree, Logistic Regression

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors = 21,weights = 'distance', leaf_size=49)

SVM_model = SVC( C= 29 , kernel='rbf' , probability = True , gamma = 30 )

DT_model = DecisionTreeClassifier(criterion='gini', max_depth=28, min_samples_leaf= 5, min_samples_split= 2)

LR_model = LogisticRegression(C= 5, multi_class= 'auto')

In [ ]:
KNN_model = KNeighborsClassifier()

SVM_model = SVC(probability = True )

DT_model = DecisionTreeClassifier()

LR_model = LogisticRegression()

In [ ]:
ensemble=VotingClassifier(
        estimators=[('KNN',KNN_model),
                   ('SVM',SVM_model),
                   ('DT',DT_model),
                   ('LR',LR_model)]
        ,voting='soft')

# Voting 모델 안에 각 모델들을 묶어주기

In [ ]:
KNN_model.fit(x_train,y_train)
SVM_model.fit(x_train,y_train)
DT_model.fit(x_train,y_train)
LR_model.fit(x_train,y_train)
ensemble.fit(x_train,y_train)

# 각 모델들을 데이터로 학습시키기

# 교차검증 적용

In [ ]:
SKF = StratifiedKFold( n_splits = 3,  shuffle = True, random_state = 16)

In [ ]:
KNN_scores=cross_validate(KNN_model, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
SVM_scores=cross_validate(SVM_model, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
DT_scores=cross_validate(DT_model, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
LR_scores=cross_validate(LR_model, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
ensemble_scores=cross_validate(ensemble, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))

In [ ]:
print(KNN_scores)
print(SVM_scores)
print(DT_scores)
print(LR_scores)
print(ensemble_scores)

In [ ]:
KNN_f1_score = KNN_scores["test_score"].mean()
SVN_f1_score = SVM_scores["test_score"].mean()
DT_f1_score = DT_scores["test_score"].mean()
LR_f1_score = LR_scores["test_score"].mean()
ensemble_f1_score = ensemble_scores["test_score"].mean()


print("f1_score: {0: .4f}".format(KNN_f1_score))
print("f1_score: {0: .4f}".format(SVN_f1_score))
print("f1_score: {0: .4f}".format(DT_f1_score))
print("f1_score: {0: .4f}".format(LR_f1_score))
print("f1_score: {0: .4f}".format(ensemble_f1_score))

# 결과 boxplot

In [ ]:
results = []
names = []

models = []
models.append(('KNN', KNN_model))
models.append(('SVM', SVM_model))
models.append(('DT', DT_model))
models.append(('LR', LR_model))
models.append(('Voting',ensemble))


for name, model in models:
    kfold = StratifiedKFold( n_splits = 3,  shuffle = True, random_state = 10)
#     cv_results = cross_validate(ensemble, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
    cv_results = cross_val_score(ensemble, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
    results.append(cv_results)
    names.append(name)

fig = plt.figure()
fig.suptitle('Classifier Comparison')
ax= fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

# test 데이터 예측

In [ ]:
pred_KNN = KNN_model.predict(x_test)
pred_SVM = SVM_model.predict(x_test)
pred_DT = DT_model.predict(x_test)
pred_LR = LR_model.predict(x_test)

# 각 모델들의 f1_score 파악을 위해 모델들 예측실행

In [ ]:
y_predict= ensemble.predict(x_test)

# 적용 모델인 Voting ensemble 예측

# Test data 교차검증 boxplot

In [ ]:
results = []
names = []

models = []
models.append(('KNN', KNN_model))
models.append(('SVM', SVM_model))
models.append(('DT', DT_model))
models.append(('LR', LR_model))
models.append(('Voting',ensemble))


for name, model in models:
    kfold = StratifiedKFold( n_splits = 3,  shuffle = True, random_state = 10)
#     cv_results = cross_validate(ensemble, x_train, y_train, cv=SKF, scoring=make_scorer(f1_score))
    cv_results = cross_val_score(ensemble, x_test, y_test, cv=SKF, scoring=make_scorer(f1_score))
    results.append(cv_results)
    names.append(name)

fig = plt.figure()
fig.suptitle('Classifier Comparison')
ax= fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

# 2020 년 사고사례 데이터

# DATA 불러오기

In [16]:
accident = pd.read_excel("초량동 지형 강우량.xlsx")   # 초량지하차도 지형정보 와 당일 시간당 강우량 데이터

In [15]:
Busanaccident = accident.drop(['SLOPE_L','SLOPE_H','FLOOD','MANHOLES_RATIO'], axis = 1)  # 불필요 컬럼제거
Busanaccidentclass = accident['FLOOD']                                                  # 분류구분

KeyError: "['FLOOD'] not found in axis"

# 가중치 주기

In [ ]:
Busanaccident.rename(columns={'R_W_SUM':'Hourly_Rainfall_Weight','HOUR_RAINFALL':'Hourly_Rainfall','IMP_SUR_RATIO':'Impervious_Surface',
                              'IMP_W_SUM':'Impervious_Surface_Weight'},inplace=True)  # 컬럼명 수정

## 강우량 가중치

In [ ]:
for idx,col in enumerate(Busanaccident['Hourly_Rainfall']):
    if col <= 15:
        Busanaccident['Hourly_Rainfall_Weight'][idx] = col / (46)
    elif col > 15 and col <= 35:
        Busanaccident['Hourly_Rainfall_Weight'][idx] = col / (18 * 2**2)
    elif col > 35 and col <= 60:
        Busanaccident['Hourly_Rainfall_Weight'][idx] = col / (17 * 3**2)
    elif col > 60 and col <= 80:
        Busanaccident['Hourly_Rainfall_Weight'][idx] = col / (11 * 4**2)
    elif col > 80:
        Busanaccident['Hourly_Rainfall_Weight'][idx] = col / (6.28 * 5**2)

 ## 불투수면 가중치

In [ ]:
for idx,col in enumerate(Busanaccident['Impervious_Surface']):
    if col <= 15:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / 6.66
    elif col > 15 and col <= 25:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (20 * 2)
    elif col > 25 and col <= 40:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (13.33 * 3)
    elif col > 40 and col <= 55:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (33.33 * 4)
    elif col > 55:
        Busanaccident['Impervious_Surface_Weight'][idx] = col / (26.67 * 5)

 ## 경사도 가중치

In [ ]:
Busanaccident['SLOPE_AVG_Weight'] = Busanaccident['SLOPE_AVG']

In [ ]:
for idx,col in enumerate(Busanaccident['SLOPE_AVG']):
    if col <= 10:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (41.67 / 5**2 )
    elif col > 10 and col <= 15:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (25 / 4**2)
    elif col > 15 and col <= 25:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (16.67 / 3**2)
    elif col > 25 and col <= 50:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (8.33 / 2**2)
    elif col > 50:
        Busanaccident['SLOPE_AVG_Weight'][idx] = col / (8.33 / 1**2)

## 고도 가중치

In [ ]:
Busanaccident['HIGH_Weight'] = Busanaccident['HIGH']

In [ ]:
for idx,col in enumerate(Busanaccident['HIGH']):
    if col <= 20:
        Busanaccident['HIGH_Weight'][idx] = col / (79.62 / 1**2)
    elif col > 20 and col <= 40:
        Busanaccident['HIGH_Weight'][idx] = col / (10.18 / 2**2)
    elif col > 40 and col <= 60:
        Busanaccident['HIGH_Weight'][idx] = col / (4.63 / 3**2)
    elif col > 60 and col <= 80:
        Busanaccident['HIGH_Weight'][idx] = col / (2.78 / 4**2)
    elif col > 80:
        Busanaccident['HIGH_Weight'][idx] = col / (2.78 / 5**2)

In [ ]:
Busanaccident = Busanaccident.drop(['SLOPE_AVG','Impervious_Surface','Hourly_Rainfall','HIGH'], axis = 1)
Busanaccident

In [ ]:
scaler = MinMaxScaler()
scaler.fit(Busanaccident)
Busanaccident = scaler.transform(Busanaccident)                         # sccler 적용

# PCA 적용

In [ ]:
pca = PCA(n_components=4)
printcipalComponents = pca.fit_transform(Busanaccident)
Busanaccident = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2','principal component3', 'principal component4'])

# 예측값 확인

In [ ]:
accident_predict= ensemble.predict(Busanaccident)                       # 위에서 학습된 모델 활용을 이용한 예측

In [ ]:
print(accident_predict)                                                 # 예측값 확인